# Main Loop

## import packages

We need some Python packages, which may or may not be installed within your conda environment. Uncomment the following lines to install them. (Don't forget to comment the lines afterwards again.)

In [1]:
# TBD
#!pip install rdflib etc.

Import packages:

In [2]:
import time
from shutil import copyfile
import os
import xml.etree.ElementTree as ET
from rdflib import Graph
import owlrl

from gpdscl.src.mm2ttl import mm2turtle # collectOwlEntries, searchForOntology

## The main loop

We assume that you have done some edits to your mindmap.

In order to export the mindmap simply process the steps of this main loop.

### Define project name, organize backup

In this early version 0.2 we make use of ugly, but simple scriptig techniques, including global variables.


In [3]:
# project = 'milk' # do not add the file extension here
project = 'sandbox'

Don't change these settings:

In [4]:
verbosity = 2
sourceDir = './mindmaps'
backupDir = './backups'
ontologyDir = './ontologies'

Because we will overwrite (!) the input file, we suggest to allocate time stamped backup files automatically.

In [5]:
timestr = time.strftime("%Y-%m-%dT%H-%M-%S")
projectSourceFile = f"{sourceDir}/{project}.mm"
projectBackupDir  = f"{backupDir}/{project}"
projectBackupFile = f"{projectBackupDir}/{project}_{timestr}.mm"

if not os.path.exists(projectBackupDir):
    os.mkdir(projectBackupDir)
copyfile(projectSourceFile, projectBackupFile)
print(f"backup done from {projectSourceFile} to {projectBackupFile}")

backup done from ./mindmaps/sandbox.mm to ./backups/sandbox/sandbox_2021-01-19T12-35-42.mm


### Read and update `.mm`

In [6]:
tree = ET.parse(projectSourceFile)
root = tree.getroot()
ontologyString = mm2turtle(root, "http://jbusse.de/ontologies/gpdscl", verbosity=verbosity)

collectOwlEntries: 
# ONTOLOGY, predicate        ID_685648358 2021-01-19 11:34:18

@prefix : <http://jbusse.de/ontology/mm2owl#> .
collectOwlEntries: 
# ONTOLOGY, object        ID_1936719561 2021-01-19 11:33:46

:Gewässer a owl:Class.
collectOwlEntries: 
# BY, predicate        ID_1449600205 2021-01-14 17:01:21

:Trophiestufe a owl:ObjectProperty .
collectOwlEntries: 
# BY, object        ID_1260890827 2021-01-15 12:45:53

:ID_1260890827 a owl:Class ;
            rdfs:subClassOf :Gewässer .
collectOwlEntries: 
# SOME, predicate        ID_1974595102 2021-01-14 17:01:21

:SOME_:Trophiestufe_:oligotroph a owl:Class ;
        owl:equivalentClass [ a owl:Restriction ;
            owl:onProperty :Trophiestufe ;
            owl:someValuesFrom :oligotroph ] .

    :Gewässer_AND_:SOME_:Trophiestufe_:oligotroph a owl:Class ;
        rdfs:subClassOf :ID_1260890827 ;
        owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( :Gewässer :SOME_:Trophiestufe_:oligotroph ) ] .

    # ow

### write files

Overwrite (!) the source file:

In [7]:
tree.write(projectSourceFile)

Export ontology:

In [8]:
with open(f"{ontologyDir}/{project}.ttl", 'w') as out:
    out.write(ontologyString)

re-read the ttl file we have exported in the preceding step. (We read it two times in order to get two completely distinct graphs).

In [9]:
g = Graph()
g.parse(f"{ontologyDir}/{project}.ttl", format='ttl')

<Graph identifier=Nbcbf9a74b3bd49a1b5de777140a1f40b (<class 'rdflib.graph.Graph'>)>

In [10]:
g2 = Graph()
g2.parse(f"{ontologyDir}/{project}.ttl", format='ttl')
owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples = False).expand(g2)

In [11]:
print(f"# triples before/after inferencing deductive closure with owlrl:\n{len(g)} / {len(g2)}")

# triples before/after inferencing deductive closure with owlrl:
83 / 399


In [12]:
g.serialize(destination  = f"{ontologyDir}/{project}_rdflib.ttl", format='turtle')
g2.serialize(destination = f"{ontologyDir}/{project}_rdflib_inferred.ttl", format='turtle')